In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct

# import some data to play with
digits = datasets.load_digits()


#choose a seed
seed = 0
np.random.seed(seed)
# Load data into train set and test set
digits = datasets.load_digits()
X = digits.data
y = np.array(digits.target, dtype = int)
X,y = shuffle(X,y)


def svmsubgradient(Theta, x, y):
#  Returns a subgradient of the objective empirical hinge loss
#
# The inputs are Theta, of size n-by-K, where K is the number of classes,
# x of size n, and y an integer in {0, 1, ..., 9}.
    G = np.zeros(Theta.shape)
    ## IMPLEMENT THE SUBGRADIENT CALCULATION -- YOUR CODE HERE
    K = Theta.shape[1]
    x.shape = d,1 
    for j in range(K):
        if y != j:
            indices = list(range(K))
            indices.remove(j)
            indices.remove(y)
            if 1+np.dot(x.T, Theta[:,[j]]-Theta[:,[y]])>np.max(1+np.matmul(x.T, Theta[:,indices]-np.tile(Theta[:,[y]],(1,8))), axis=1):
                G[:,[j]] = x
            # Else zero
        else:
            indices = list(range(K))
            indices.remove(y)
            if np.max(1+np.matmul(x.T, Theta[:,indices]-np.tile(Theta[:,[y]],(1,9))), axis=1)>0:
                G[:,[j]] = -x
            # Else zero
    
    return(G)

def sgd(Xtrain, ytrain, maxiter = 10, init_stepsize = 1.0, l2_radius = 10000):
#
# Performs maxiter iterations of projected stochastic gradient descent
# on the data contained in the matrix Xtrain, of size n-by-d, where n
# is the sample size and d is the dimension, and the label vector
# ytrain of integers in {0, 1, ..., 9}. Returns two d-by-10
# classification matrices Theta and mean_Theta, where the first is the final
# point of SGD and the second is the mean of all the iterates of SGD.
#
# Each iteration consists of choosing a random index from n and the
# associated data point in X, taking a subgradient step for the
# multiclass SVM objective, and projecting onto the Euclidean ball
# The stepsize is init_stepsize / sqrt(iteration).
    K = 10
    NN, dd = Xtrain.shape
    Theta = np.zeros(dd*K)
    Theta.shape = dd,K
    mean_Theta = np.zeros(dd*K)
    mean_Theta.shape = dd,K
    ## YOUR CODE HERE -- IMPLEMENT PROJECTED STOCHASTIC GRADIENT DESCENT
    
    for iteration in range(maxiter):
        epsilon_k = init_stepsize/np.sqrt(iteration+1)
        n = np.random.choice(NN)
        x = Xtrain[n,:]
        y = ytrain[n]
        Theta = Theta - epsilon_k * svmsubgradient(Theta, x, y)
        if np.sum(np.square(Theta))>l2_radius**2:
            Theta = Theta/np.sum(np.square(Theta))*l2_radius
        mean_Theta = (iteration*mean_Theta+Theta)/(iteration+1)
    return Theta, mean_Theta

def Classify(Xdata, Theta):
#
# Takes in an N-by-d data matrix Adata, where d is the dimension and N
# is the sample size, and a classifier X, which is of size d-by-K,
# where K is the number of classes.
#
# Returns a vector of length N consisting of the predicted digits in
# the classes.
    scores = np.matmul(Xdata, Theta)
    inds = np.argmax(scores, axis = 1)
    return(inds)

N,d = X.shape
def main(trainsize):
    N,d = X.shape
    Ntest = int(100)
    Ntrain = int(trainsize)
    M = Ntest + Ntrain
    Xtrain = X[0:Ntrain,:]
    ytrain = y[0:Ntrain]
    Xtest = X[Ntrain:M,:]
    ytest = y[Ntrain:M]
    l2_radius = 40.0
    M_raw = np.sqrt(np.mean(np.sum(np.square(Xtrain))))
    init_stepsize = l2_radius/M_raw
    maxiter = 40000
    Theta, mean_Theta = sgd(Xtrain, ytrain, maxiter, init_stepsize, l2_radius)
    print('Error rate')
    print(np.sum(np.not_equal(Classify(Xtest, mean_Theta),ytest)/Ntest))
    return 


In [2]:
error_rates = list()
for trainsize in [20,50,100,500,1000,1500]:
    error_rates.append(main(trainsize))

Error rate
0.43000000000000005
Error rate
0.25
Error rate
0.15000000000000002
Error rate
0.05
Error rate
0.05
Error rate
0.04
